# Praxisübung: Vorhersage von Wohnungsmieten
Ziel dieser Übung ist es, die Monatskaltmiete für eine Mietwohnung vorherzusagen anhand verschiedener Features wie Größe und Zustand der Wohnung, Nebenkosten, Bundesland etc. Laden Sie sich dafür die Dateien "immo_data.csv" und "immo_data_column_description.csv" herunter. Alternativ können Sie sich die Datei immo_data.csv hier herunterladen: https://www.kaggle.com/corrieaar/apartment-rental-offers-in-germany (Account benötigt).

Die Daten stammen von Immoscout24, der größten Immobilienplattform in Deutschland. Immoscout24 bietet sowohl Miet- als auch Kaufobjekte an, allerdings enthalten die Daten nur Angebote für Mietobjekte. Zu einem bestimmten Zeitpunkt wurden alle verfügbaren Angebote von der Website abgefragt und gespeichert. Dieser Vorgang wurde dreimal wiederholt, so dass der Datensatz Angebote aus den Zeiträumen 2018-09-22, 2019-05-10 und 2019-10-08 enthält.

Der Datensatz enthält die meisten wichtigen Eigenschaften, wie z.B. die Größe der Wohnfläche, die Miete, sowohl Kaltmiete als auch Gesamtmiete (falls zutreffend), die Lage (Straße und Hausnummer, falls vorhanden, Postleitzahl und Bundesland), Energieart usw. Außerdem gibt es zwei Variablen, die längere Freitextbeschreibungen enthalten: Beschreibung mit einem Text, der das Angebot beschreibt, und Ausstattung mit einer Beschreibung aller verfügbaren Einrichtungen, der neuesten Renovierung usw. Die Datumsspalte wurde hinzugefügt, um den Zeitpunkt des Scrapings anzugeben.

Wir möchten unser Wissen über lineare Regression und Datenanalyse an diesem Datensatz testen.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from IPython.display import display

pd.options.display.max_columns = 50

In [ ]:
df_raw = pd.read_csv("Daten/immo_data.csv")
desc = pd.read_csv("Daten/immo_data_column_description.csv")

Spaltenbeschreibungen

In [ ]:
desc

Aus der Liste der Beschreibungen fallen uns einige Attribute auf, die doppelt auftreten oder (wahrscheinlich) keine Vorhersagekraft haben (eigentlich müsste man das verifizieren!). Diese schließen wir direkt aus, indem wir die "drop" Methode des DataFrame aufrufen und die entsprechenden Spaltennamen übergeben. Das Argument errors="ignore" besagt, dass, falls die Spalte nicht (mehr) existiert, keine Fehlermeldung produziert werden soll. Das ist hier hilfreich: falls man im Notebook eine Zelle mehrmals ausführt, würde ab dem zweiten Mal immer eine Fehlermeldung auftauchen.

In [ ]:
df = df_raw.copy()
# Einschränken des Datensatzes auf eine bestimmte Region
#df = df[df["regio2"]=="Karlsruhe"]
#df = df[df["regio1"]=="Baden_Württemberg"]

df = df.drop(["scoutId", "houseNumber", "geo_bln", "geo_krs", "date"], axis=1, errors="ignore")

## Datenanalyse

### Nicht-numerische Daten
Wir schauen uns die Spalten, d.h. die *Attribute*, von denen einige später zu *Features* werden, genauer an. Um es etwas übersichtlicher zu halten schauen wir uns zunächst die nicht-numerischen und dann die numerischen Spalten an.

In [ ]:
numeric_columns = df.select_dtypes(include=np.number).columns
categorical_columns = df.select_dtypes(exclude=np.number).columns

df[categorical_columns].describe()

Wir erinnern uns: Wenn kategorielle Features mittels OneHot Encoding transformiert werden, so entsteht für jede Ausprägung eine neue Spalte. Die Anzahl der unterschiedlichen Ausprägungen gibt die Zeile "unique" an. Wenn wir also z.B. "description" (das ist ein Freitext Feld) mittels OneHot Encoding transformieren würden, würden wir einen DataFrame (also eine Datenmatrix) mit über 200,000 Spalten erhalten! Attribute mit zu vielen Ausprägungen schließen wir also zunächst aus.

In [ ]:
df = df.drop(["street", "streetPlain", "description", "facilities"], axis=1, errors="ignore")

Uns fällt ein Attribut "firingTypes" (laut Beschreibung die "main energy source") auf, das 132 verschiedene Ausprägungen hat. Das schauen wir uns genauer an.

In [ ]:
df["firingTypes"].unique()

Es gibt ein ähnliches Attribut heatingType, das hat aber deutlich weniger Ausprägungen. Wir schauen uns an, welche Ausprägungen es gibt.

In [ ]:
df["heatingType"].unique()

FiringTypes enthält *Kombinationen* von Ausprägungen von heatingTypes, getrennt durch Doppelpunkte. Wir entscheiden uns, das detailliertere von beiden Attributen auszuschließen, damit das entstehende Datenset nicht so groß wird.

In [ ]:
df = df.drop("firingTypes", axis=1, errors="ignore")

Wir schauen noch einmal die kategoriellen Attribute an, die wir als Kandidaten für Features behalten möchten.

In [ ]:
display(df.describe(exclude=np.number))

Als nächstes schauen wir uns noch die Häufigkeitsverteilung der verschiedenen Ausprägungen an. Hier arbeiten wir mit der Methode .value_counts() von Pandas und erzeugen jeden Plot separat in einem Gitter.

In [ ]:
categorical_columns = df.select_dtypes(exclude=np.number).columns

# n times n subplots
n = int(np.ceil(np.sqrt(len(categorical_columns))))
fig, axes = plt.subplots(nrows=n, ncols=n)
fig.tight_layout()

for i, col in enumerate(categorical_columns):
    plt.subplot(n, n, i+1)
    if col in ["regio2", "regio3"]:  # regio2 und regio3 haben zu viele Ausprägungen
        pass
    else:
        df[col].value_counts().plot(kind='bar', figsize=(20,20), title=col)

## ✏ Aufgabe
Beschreiben Sie die Verteilungen.

### Numerische Daten

Als nächstes schauen wir uns nun die numerischen Attribute genauer an.

In [ ]:
display(df.describe(include=np.number))

Wir sehen, dass es einige extreme *Ausreißer* in den Daten gibt. So gibt es z.B. eine Wohnung mit monatlichen Nebenkosten von 146118 EUR, ein Objekt, das etwa 15 mio EUR Miete pro Monat kostet oder 111111m² Wohnfläche besitzt. Wir speichern uns die Namen der verdächtigen Spalten in einer Liste.
Weiterhin sehen wir, dass das Feature telekomHybridUploadSpeed immer den Wert 10 hat oder leer ist, deshalb löschen wir es.

In [ ]:
interesting_columns = ["serviceCharge", "totalRent", "yearConstructed", "noParkSpaces", "baseRent", "livingSpace", "noRooms", "numberOfFloors", "heatingCosts", "lastRefurbish"]
df.drop(["telekomHybridUploadSpeed"], axis=1, inplace=True, errors="ignore")

Wir schließen explizit die Postleitzahl aus.

## ✏ Aufgabe
Glauben Sie, dass die PLZ ein gutes Feature wäre, d.h. hätte die PLZ Aussagekraft bezüglich der Kaltmiete?

In [ ]:
df = df.drop(["regio3","geo_plz"], axis=1, errors="ignore")

### Untersuchung von Ausreißern

In [ ]:
# plotly dauert lange, da der Datensatz groß ist
#fig = make_subplots(rows=12, cols=1, subplot_titles=interesting_columns)
#fig.update_layout(width=1200, height=1200)

#for i, col in enumerate(interesting_columns):
#    fig.add_trace(
#        go.Box(x=df[col]),
#        #row=i//4+1, col=i%4+1#
#        row=i+1, col=1
#    )
#fig.show()

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(20,20))
fig.tight_layout()

for i, col in enumerate(interesting_columns):
    plt.subplot(3, 4, i+1)
    df[col].plot(kind="box")

## Datenbereinigung
Das sind zu viele Ausreißer, als dass wir sie alle gesondert betrachten möchten. Wir schauen uns an, was passieren würde, wenn wir einfach die unteren und oberen 0.5% der Daten bezüglich dieser Attribute wegwerfen würden.

In [ ]:
display(df[interesting_columns].quantile(0.995))
display(df[interesting_columns].quantile(0.005))

Das sieht nach einer gesünderen Verteilung aus. Wir wollen mit diesen Daten weiterarbeiten. Dazu erzeugen wir eine Kopie von df. Sonst würde df bei versehentlichem mehrmaligen Ausführen der Zelle immer kleiner werden (die Quantile würden auf dem neuen, kleineren df neu berechnet)

In [ ]:
upper_limits = df[interesting_columns].quantile(0.995)
lower_limits = df[interesting_columns].quantile(0.005)

df_reduced = df.copy()
display(df_reduced.describe(include=np.number))

# Für jede Spalte behalten wir: Daten die < (99.5%-Quantil) sind und > (0.5%-Quantil) sind ODER die NaN sind (damit befassen wir uns spaeter noch) 
for col in interesting_columns:
    df_reduced = df_reduced[((df_reduced[col] <= upper_limits[col]) & (df_reduced[col] >= lower_limits[col])) | df_reduced[col].isna()]

Wir haben nun also eine grobe Auswahl an Attributen getroffen, mit denen wir ab jetzt weiterarbeiten möchten.

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(20,20))
fig.tight_layout()

for i, col in enumerate(interesting_columns):
    plt.subplot(3, 4, i+1)
    df_reduced[col].plot(kind="box")

### Behandlung von fehlenden Datensätzen
Wir haben eben schon gesehen, dass es einige NaN (=not a number) Felder in den Daten gibt. Bevor wir entscheiden, was wir damit tun, verschaffen wir uns einen Überblick, wie viele das etwa sind.

In [ ]:
display(df_reduced.isna().sum()) # isna() wandelt jeden Wert in einen Boolschen Datentyp (0/1) um. sum() summiert diese Werte pro Spalte. Man erhält also die Anzahl der NaN Werte pro Spalte.

Das sind z.T. sehr viele NaN Werte. Wenn wir alle Daten ignorieren würden, würden nicht mehr viele übrig bleiben. Wir verfolgen zunächst folgende Strategie: 
- Für numerische Werte wird für einen fehlenden Wert der Mittelwert der restlichen Werte eingesetzt.
- Für nicht-numerische Werte wird für einen fehlenden Wert der häufigsten Wert (=Modus) der restlichen Werte eingesetzt.

Zum Auffüllen der Werte stellt scikit-learn stellt die Klasse SimpleImputer bereit.

In [ ]:
from sklearn.impute import SimpleImputer

# Technisches Detail: wir müssen noch die Bool-Werte explizit konvertieren, sonst wirft der Imputer einen Fehler
df_reduced = df_reduced.replace({False: 0, True: 1})

numeric_columns = df_reduced.select_dtypes(include=np.number).columns
numeric_columns = numeric_columns.drop("baseRent") # Wenn das Label fehlt, wird es nicht ersetzt
categorical_columns = df_reduced.select_dtypes(exclude=np.number).columns

imp_freq = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')

df_reduced.loc[:,numeric_columns] = imp_mean.fit_transform(df_reduced[numeric_columns])
df_reduced.loc[:,categorical_columns] = imp_freq.fit_transform(df_reduced[categorical_columns])

## Feature Engineering
### Korrelationen
Wir erzeugen uns Scatter Plots um offensichtlich lineare oder nichtlineare Zusammenhänge zwischen einzelnen Features und der Zielvariable zu identifizieren.

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(20,20))
fig.tight_layout()

for i, col in enumerate(interesting_columns):
    #plt.subplot(4, 3, i+1)
    df_reduced[["baseRent",col]].plot.scatter(x=col, y="baseRent", ax=axes[i//3,i%3], alpha=0.2)

## ✏ Aufgabe
Schauen Sie sich die Korrelationen an. Was fällt Ihnen auf?

### OneHot Encoding
Als nächstes transformieren wir die kategoriellen Spalten mittels OneHot-Encoding. Dafür benutzen wir die Pandas Funktion get_dummies() die in einem DataFrame für jede kategorielle Spalte das OneHot Encoding durchführt. D.h., für jede kategorielle Spalte entstehen mehrere 0/1-wertige Spalten, und zwar gerade so viele wie es Ausprägungen der Variable gibt. Das Ergebnis ist nun ein DataFrame, der weitaus mehr Spalten hat. Diese sind dafür aber alle numerisch, d.h. wir können sie für die lineare Regression (oder einen anderen ML Algorithmus verwenden).

In [ ]:
df_reduced = pd.get_dummies(df_reduced)

Ein letzter Blick in die Daten ob alles funktioniert hat:

In [ ]:
display(df_reduced)

### Entfernen des Labels aus den Attributen
WICHTIG: Wir möchten die Kaltmiete bzw. Gesamtmiete *vorhersagen*. Diese (und das davon direkt abgeleitete Attribut baseRentRange) müssen wir also auch noch ausschließen. Zuvor entfernen wir noch Datensätze, für die wir kein Label haben.

In [ ]:
df_reduced = df_reduced[df_reduced["baseRent"].isna() == False] # Entfernen der Datensätze ohne Label
y = df_reduced["baseRent"]
df_reduced = df_reduced.drop(["baseRent", "totalRent","baseRentRange"], axis=1, errors="ignore") 

### Erstellen von Trainings- und Testset
Bevor wir die Daten weiter bearbeiten, spalten wir sie in Trainings- und Testset auf.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_reduced, y, test_size=0.2, random_state=1)

### Modelltraining
Nun ist es soweit: Wir trainieren das Regressionsmodel!

In [ ]:
from sklearn import linear_model
m = linear_model.LinearRegression()
m.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
r2_train = m.score(X_train, y_train)
mse_train = mean_squared_error(y_train, m.predict(X_train))
mae_train = mean_absolute_error(y_train, m.predict(X_train))
r2_test = m.score(X_test, y_test)
mse_test = mean_squared_error(y_test, m.predict(X_test))
mae_test = mean_absolute_error(y_test, m.predict(X_test))

In [ ]:
print(f"R² Trainingsdaten: {r2_train}")
print(f"Mean squared error Trainingsdaten: {mse_train}")
print(f"Mean absolute error Trainingsdaten: {mae_train}")
print(f"R² Testdaten: {r2_test}")
print(f"Mean squared error Testdaten: {mse_test}")
print(f"Mean absolute error Testdaten: {mae_test}")

Das Ergebnis der multiplen linearen Regression ist nicht so einfach zu visualisieren. Hier ein Versuch als Scatterplot zwischen der Größe der Wohnung und der Kaltmiete (blau) bzw. der vorhergesagten Kaltmiete (rot).

In [ ]:
plt.plot(X_test["livingSpace"], y_test, "bo", alpha=0.2)
plt.plot(X_test["livingSpace"], m.predict(X_test), "ro", alpha=0.2)

## ✏ Aufgabe 1
Arbeiten Sie das Notebook durch und versuchen Sie die einzelnen Schritte nachzuvollziehen.

## ✏ Aufgabe 2
Wie ändert sich die Vorhersagequalität des Modells, wenn sie die Ausreißer nicht entfernen?
Gehen Sie dazu zum Abschnitt Datenbereinigung, identifizieren sie den Code, der für das Entfernen der Ausreißer zuständig ist und kommentieren sie ihn aus.

## ✏ Aufgabe 3
Erstellen Sie ein Modell, das die Kaltmiete nur für eine bestimmte Region (z.B. Baden Württemberg, Karlsruhe, Karlsruhe und benachbarte Landkreise) vorhersagt. 
Am einfachsten geht das, indem Sie den Datensatz direkt nach dem Einlesen der Daten einschränken und dann alle weiteren Zellen unverändert ausführen. Wie ist die Vorhersagequalität auf Trainings- und Testdaten?

## ✏ Aufgabe 4
Erstellen Sie ein Modell mit anderen Features. Hier einige Vorschläge:
- Nehmen Sie Features mit in das Modell auf, die wir am Anfang ausgeschlossen hatten und schauen sie, ob sie die Vorhersagegüte verbessern.
- Nehmen Sie weniger Features. Die Vorhersagegüte (zumindest auf dem Trainingsdatensatz) wird dadurch zwar schlechter, doch auf kleinen Datensätzen sollten Sie ein robusteres Modell erhalten, d.h. eines, welches auf den Testdaten ähnlich gute Ergebnisse liefert wie auf den Trainingsdaten. Entweder wählen Sie per Hand geeignete Features aus oder Sie nehmen z.B. eine automatische Methode wie SelectKBest. Benutzen Sie dazu folgenden Code:

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression
feature_selection = SelectKBest(f_regression, k=50) # Ein Transformer, der per f_regression die 50 vielversprechendsten Features auswählt

X_train = feature_selection.fit_transform(X_train, y_train)
X_test = feature_selection.transform(X_test)

- Erstellen Sie sich neue Features durch Kombination oder Umformung vorhandener Features, z.B. polynomielle bzw. Interaktionsfeatures. Hier ein Code Beispiel, mit dem sie sich Interaktionsfeatures Bundesland x livingSpace erzeugen. Anschaulich bedeutet das: Der Koeffizient für das Feature livingSpace kann von Bundesland zu Bundesland unterschiedlich sein. Der Koeffizient für livingSpace ist anschaulich der Preis, der für einen zusätzlichen qm Wohnung mehr bezahlt werden muss.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
pf = PolynomialFeatures(degree=2,interaction_only=True) # Ein Transformer, der Interaktionsfeatures vom (bis zu) Grad 2 erzeugt

# Interaktionsfeatures: Bundesland x livingSpace
bundeslaender_columns = "regio1_" + df["regio1"].unique()
for col in bundeslaender_columns:
    features = pf.fit_transform(df_reduced[[col,"livingSpace"]])
    df_reduced[col + "_livingSpace"] = features[:,-1]

## ✏ Aufgabe 5 🤯
  - Sagen sie die Kaltmiete für Ihre Wohnung voraus. Überlegen Sie sich dazu, wie der Featurevektor für Ihre Wohnung aussehen würde, erstellen Sie einen DataFrame mit einer Zeile und wenden Sie das gelernte Modell mit .predict() an.
  - Wie würde die Normalengleichung für das Problem aussehen? Versuchen Sie, statt des scikit-learn Modells das Modell "per Hand" zu trainieren, indem Sie die Normalengleichung aufstellen und nach den Modellparametern $w$ auflösen (z.B. mit numpy.linalg.solve). Machen Sie sich dazu zunächst die Problemdimensionen klar in Analogie zu den Formeln aus der Vorlesung.